<a href="https://colab.research.google.com/github/specM7/DSGP_Group_33_Brain_Tumor_Predictor/blob/main/Meningioma_Adrian_2425482.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧠 AI-Powered Brain Tumor Predictor for Meningioma(Adrian)


In [53]:
# Core
import os, glob
import numpy as np

# Image processing
import cv2
from google.colab.patches import cv2_imshow

# Deep learning
import tensorflow as tf
from tensorflow.keras import layers, models

# Visualization
import matplotlib.pyplot as plt

print("TensorFlow:", tf.__version__)


TensorFlow: 2.19.0


In [54]:
from google.colab import files

uploaded = files.upload()        # choose DataSet.zip from your PC

# Unzip into /content
!unzip -q DataSet.zip -d /content
!ls /content


Saving DataSet.zip to DataSet (4).zip
error [DataSet.zip]:  missing 54525952 bytes in zipfile
  (attempting to process anyway)
error: invalid zip file with overlapped components (possible zip bomb)
'DataSet (1).zip'  'DataSet (3).zip'   DataSet_pre   sample_data
'DataSet (2).zip'  'DataSet (4).zip'   DataSet.zip


In [55]:
print("Folders under /content:")
!ls /content

print("\nDirectories containing 'Training':")
!find /content -maxdepth 5 -type d -iname "Training"


Folders under /content:
'DataSet (1).zip'  'DataSet (3).zip'   DataSet_pre   sample_data
'DataSet (2).zip'  'DataSet (4).zip'   DataSet.zip

Directories containing 'Training':
/content/DataSet_pre/Training


In [57]:
print("Folders under /content:")
!ls /content

print("\nAny folders that contain 'Training':")
!find /content -maxdepth 5 -type d -iname "Training"


Folders under /content:
'DataSet (1).zip'  'DataSet (3).zip'   DataSet_pre   sample_data
'DataSet (2).zip'  'DataSet (4).zip'   DataSet.zip

Any folders that contain 'Training':
/content/DataSet_pre/Training


In [56]:
import os, glob, cv2
import numpy as np

# >>> CHANGE THIS LINE based on Cell 3 output <<<
raw_root = "/content/archive"   # example; replace with your actual root

print("RAW training folders:", os.listdir(f"{raw_root}/Training"))

pre_root = "/content/DataSet_pre"

def preprocess_img(path, size=(224, 224)):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise ValueError(f"Could not read image: {path}")
    img = cv2.resize(img, size)
    img = cv2.GaussianBlur(img, (5, 5), 0)
    img = cv2.equalizeHist(img)
    img = img.astype("float32") / 255.0
    return img


FileNotFoundError: [Errno 2] No such file or directory: '/content/archive/Training'

In [ ]:
# Recreate pre_root
!rm -rf "$pre_root"
classes = os.listdir(f"{raw_root}/Training")
print("Classes:", classes)

for split in ["Training", "Testing"]:
    for cls in classes:
        in_dir = os.path.join(raw_root, split, cls)
        out_dir = os.path.join(pre_root, split, cls)
        os.makedirs(out_dir, exist_ok=True)

        img_paths = glob.glob(os.path.join(in_dir, "*"))
        print(f"{split} / {cls}: found {len(img_paths)} raw images")

        for path in img_paths:
            img = preprocess_img(path)
            base = os.path.basename(path)
            name, _ = os.path.splitext(base)
            save_name = name + ".jpg"
            cv2.imwrite(os.path.join(out_dir, save_name),
                        (img * 255).astype("uint8"))

print("\nAfter preprocessing:")
print("Training classes:", os.listdir(f"{pre_root}/Training"))
import glob as _g
print("meningioma files:",
      len(_g.glob(f"{pre_root}/Training/meningioma/*")))


In [ ]:
img_size = (224, 224)
batch_size = 32

train_dir = f"{pre_root}/Training"
test_dir  = f"{pre_root}/Testing"

train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=img_size,
    batch_size=batch_size,
    validation_split=0.2,
    subset="training",
    seed=42,
    color_mode="grayscale"
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=img_size,
    batch_size=batch_size,
    validation_split=0.2,
    subset="validation",
    seed=42,
    color_mode="grayscale"
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=img_size,
    batch_size=batch_size,
    shuffle=False,
    color_mode="grayscale"
)

print("Classes:", train_ds.class_names)

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.shuffle(1000).prefetch(AUTOTUNE)
val_ds   = val_ds.prefetch(AUTOTUNE)
test_ds  = test_ds.prefetch(AUTOTUNE)


In [ ]:
model = models.Sequential([
    layers.Rescaling(1./255, input_shape=img_size + (1,)),
    layers.Conv2D(32, 3, activation="relu"),
    layers.MaxPooling2D(),

    layers.Conv2D(64, 3, activation="relu"),
    layers.MaxPooling2D(),

    layers.Conv2D(128, 3, activation="relu"),
    layers.MaxPooling2D(),

    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(4, activation="softmax")   # 4 classes
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=25
)

test_loss, test_acc = model.evaluate(test_ds)
print("Test accuracy:", test_acc)
